**Classification du texte en utilisant BERT**

In [ ]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00


Import des bibliothèques nécessaires

In [ ]:
import pandas as pd
import numpy as np
import os
import random
import matplotlib.pyplot as plt
from datasets import load_dataset
import torch
import torch.nn as nn
from transformers import AutoModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, SequentialSampler

Import des données

In [ ]:
data = load_dataset('ag_news')
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

Analyse exploratoire des données

In [ ]:
df = data['train']
df = pd.DataFrame(df)

In [ ]:
df[df['label']==0].head()

,text,label
492,Venezuelans Vote Early in Referendum on Chavez...,0
493,S.Koreans Clash with Police on Iraq Troop Disp...,0
494,Palestinians in Israeli Jails Start Hunger Str...,0
495,Seven Georgian soldiers wounded as South Osset...,0
496,Rwandan Troops Arrive in Darfur (AP) AP - Doze...,0


In [ ]:
df = pd.DataFrame(data['train'])
df.head()

,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2


In [ ]:
df[df['label']==1].head()

,text,label
448,"Phelps, Thorpe Advance in 200 Freestyle (AP) A...",1
449,Reds Knock Padres Out of Wild-Card Lead (AP) A...,1
450,"Dreaming done, NBA stars awaken to harsh Olymp...",1
451,"Indians Beat Twins 7-1, Nearing AL Lead (AP) A...",1
452,"Galaxy, Crew Play to 0-0 Tie (AP) AP - Kevin H...",1


In [ ]:
df['label'].value_counts()

2    30000
3    30000
1    30000
0    30000
Name: label, dtype: int64

In [ ]:
num_train_samples = 10000
num_new_samples = int(num_train_samples/df['label'].nunique())

# Regrouper par 'label' et échantillonner un nombre égal de lignes de chaque groupe
df = df.groupby('label', group_keys=False).apply(lambda x: x.sample(min(len(x), num_new_samples)))

# Display the resulting DataFrame
df['label'].value_counts()

0    2500
1    2500
2    2500
3    2500
Name: label, dtype: int64

Prétraitement des données

In [ ]:
# Chargement du tokenizer BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
#longueur maximale
max_sent_len = -1
for i in range(num_train_samples):
  max_sent_len = max(max_sent_len, len(tokenizer.encode(df['text'].iloc[i], add_special_tokens=True)))
print(f'Max_length of Input sentences : {max_sent_len}')

# Récupération des phrases et des étiquettes
sentences = df.text.values
labels = df.label.values

Max_length of Input sentences : 358


In [ ]:
# Démarrage de la tokenization et du prétraitement des données
input_ids = []  # Liste pour stocker les ids de tokens de chaque phrase
attention_masks = []  # Liste pour stocker les masques d'attention de chaque phrase

# Parcourir chaque phrase dans le DataFrame et les tokeniser
for sent in sentences:
    # Utiliser la méthode encode_plus du tokenizer pour encoder la phrase et créer le masque d'attention
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens=True,   # Ajouter les tokens spéciaux [CLS] et [SEP]
                        max_length=max_sent_len,   # Longueur maximale de la phrase
                        padding='max_length',      # Remplissage pour avoir une longueur fixe
                        truncation=True,          # Tronquer les phrases si nécessaire
                        return_attention_mask=True,   # Créer les masques d'attention
                        return_tensors='pt',       # Retourner des tensors pytorch
                   )

    # Ajouter les identifiants de tokens et les masques d'attention à leurs listes respectives
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Conversion des listes en tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Affichage d'un exemple
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])


Original:  Sup on a new alphabet soup AS A responsible nuclear power, India is ready to work with like-minded countries in strengthening the global non-proliferation system.
Token IDs: tensor([  101, 10514,  2361,  2006,  1037,  2047, 12440, 11350,  2004,  1037,
         3625,  4517,  2373,  1010,  2634,  2003,  3201,  2000,  2147,  2007,
         2066,  1011, 13128,  3032,  1999, 16003,  1996,  3795,  2512,  1011,
        20250,  2291,  1012,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,

In [ ]:
# Création du TensorDataset à partir des identifiants de tokens, des masques d'attention et des labels
dataset = TensorDataset(input_ids, attention_masks, labels)

# Utilisation de la règle 90-10 pour diviser les données en ensembles d'entraînement et de validation
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

9,000 training samples
1,000 validation samples


In [ ]:
# Définition de la taille des lots (batch_size)
batch_size = 16

# Création des DataLoader pour l'ensemble d'entraînement et l'ensemble de validation
train_loader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),  # RandomSampler pour mélanger les données à chaque époque
            batch_size = batch_size
        )

validation_loader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),  # SequentialSampler pour parcourir les données séquentiellement
            batch_size = batch_size
        )


Création d'une instance du modèle BertForSequenceClassification

In [ ]:
# Chargement du modèle BertForSequenceClassification pré-entrainé
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=4,
    output_attentions=True,
    output_hidden_states=True,
    use_cache=True
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Définition de l'optimiseur
optimizer = AdamW(model.parameters(),lr = 6e-5, eps = 1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Détecte si un GPU est disponible et le sélectionne pour les calculs, sinon utilise le CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Transférer le modèle sur le device
input_ids = input_ids.to(device)
attention_masks = attention_masks.to(device)
labels = labels.to(device)
model = model.to(device)

Entrainement du modèle

In [ ]:
# Entraînement du modèle
from transformers import get_linear_schedule_with_warmup
epochs = 4
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

for epoch_i in range(0, epochs):
    total_train_loss = 0
    model.train() # Mettre le modèle en mode d'entraînement

    for step, batch in enumerate(train_loader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        optimizer.zero_grad()    # Réinitialiser les gradients à zéro

        output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = output.loss  # Calcul de la perte
        total_train_loss += loss.item()  # Ajouter la perte du lot à la perte totale

        loss.backward()   # Calcul des gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Clippe les gradients pour éviter l'explosion du gradient
        optimizer.step()   #Mettre à jour les poids du modèle
        scheduler.step()   # Mettre à jour le scheduler

    avg_train_loss = total_train_loss / len(train_loader)

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print("  Average training loss: {0:.2f}".format(avg_train_loss))


======== Epoch 1 / 4 ========
  Average training loss: 0.37

======== Epoch 2 / 4 ========
  Average training loss: 0.20

======== Epoch 3 / 4 ========
  Average training loss: 0.11

======== Epoch 4 / 4 ========
  Average training loss: 0.05


Evaluation du modèle


In [ ]:
# Évaluation du modèle
model.eval() # Mettre le modèle en mode d'évaluation

# Variables pour suivre le nombre total de prédictions correctes et le nombre total d'échantillons
total_correct = 0
total_samples = 0

with torch.no_grad(): # On ne calcule pas les gradients pendant l'évaluation
    for batch in validation_loader:
        # Transférer les données sur le device
        input_ids = batch[0].to(device)
        attention_masks = batch[1].to(device)
        labels = batch[2].to(device)

        outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)
        _, predicted = torch.max(outputs.logits, dim=1)  # Obtenir les prédictions en sélectionnant l'indice de la valeur maximale

        total_samples += labels.size(0)  # Mettre à jour le nombre total d'échantillons
        total_correct += (predicted == labels).sum().item() # Mettre à jour le nombre total de prédictions correctes

accuracy = total_correct / total_samples
print('Accuracy:', accuracy)


Accuracy: 0.914
